This script loads raw neuropixel spike times and prepares data for further analysis by performing the following steps:
1. Select which task entries to analyze
2. Load behavioral data and select good trials based on the reach time distributions.
3. Load neuropixel spike times
    - Bin spike times
    - Align data to the even of interest
    - Smooth timeseries with a Gaussian kernel
4. Saves preprocessed data
5. Plots basic neural data figures
    - Trial averaged firing rate
    - Raster plots
    - Raster plots organized by target direction

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import aopy
import os
import pandas as pds
from db import dbfunctions as db
from ipywidgets import interactive, widgets
import scipy
import h5py
from tqdm.auto import tqdm 
import seaborn as sn
import sklearn
import multiprocessing as mp
import time
import glob
from datetime import date
from ibldsp.voltage import detect_bad_channels, interpolate_bad_channels, destripe_lfp


# Set Parameters

In [2]:
# Paths
subject = 'affi'
data_path_preproc = '/media/moor-data/preprocessed.new/'
data_path_raw = '/media/moor-data/raw/neuropixels/'
save_dir = "/media/moor-data/results/Ryan/neuropixel_targeting/np_analysis_preproc_data"
behavior_save_dir = "/media/moor-data/results/Ryan/neuropixel_targeting/behavior"
lfp_power_save_dir = f"/media/moor-data/postprocessed/{subject}/neuropixel_lfp_power"

base_save_dir = "/media/moor-data/results/Ryan/neuropixel_targeting/"
np_preproc_data_folder = 'np_analysis_preproc_data'
ecog_dec_acc_file_name = 'ecog_decoding_maps/npinsert_ecog_decoding'
save_lfp_power = True

# General data parameters
task_coords = 'yzx'
task_perturb = None
task_rotation = 0

# Task event code definitions
task_codes = aopy.data.bmi3d.load_bmi3d_task_codes()
CENTER_TARGET_ON = 16
CURSOR_ENTER_CENTER_TARGET = 80
CURSOR_ENTER_PERIPHERAL_TARGET = list(range(81,89))
PERIPHERAL_TARGET_ON = list(range(17,25))
CENTER_TARGET_OFF = 32
REWARD = 48
DELAY_PENALTY = 66
TIMEOUT_PENALTY = 65
HOLD_PENALTY = 64
PAUSE = 254
TIME_ZERO = 238
TRIAL_END = 239

# Trial selection parameters
trial_filter = lambda t: CENTER_TARGET_OFF in t
success_rate_window = 19
reach_time_std_thresh = 3

# Neuropixel data parameters
implant_name = ['NP_Insert72', 'NP_Insert137']
start_date = '2023-07-13'
if subject == 'beignet':
    end_date = '2024-02-05' # for beignet
else:
    start_date = '2024-05-20'
    end_date = date.today()
elec_config = 'bottom'
spike_bin_width_mc = 0.01 #[s]
smooth_width = 150
smooth_nstd = 3

# Task data selection parameters
tbefore_mc = 0.2
tafter_mc = .8

# Filter parameters
bands = [(1,4), (4,12), (12,30), (30,80), (80,200)]
n = 0.5
w = 4
n, p, k = aopy.precondition.base.convert_taper_parameters(n,w)
print(n, p, k)

# Visualization parameters
colors = sn.color_palette(n_colors=9)

0.5 2.0 3


In [3]:
def get_cursor_leave_center_time(data, samplerate, target_radius):
    '''
    Compute the time when the cursor leaves the center target radius
    
    Args:
        traj (ntrials list of (nt,2)): x,y trajectory data
        samplerate
        target_radius (float): the radius of the center target
        
    Returns:
        cursor_leave_center_time (ntrials list): the time when the cursor leaves the center target radius
    '''
    ntr = len(data)
    cursor_leave_center_time = []
    
    for itr in range(ntr):
        t_axis = np.arange(data[itr].shape[0])/samplerate
        
        dist = np.sqrt(data[itr][:,0]**2 + data[itr][:,1]**2)
        leave_idx = np.where(dist>target_radius)[0]
        temp = t_axis[leave_idx]
        cursor_leave_center_time.append(temp[0])
    
    return cursor_leave_center_time

def get_cursor_leave_center_idx(data, target_radius):
    '''
    Compute the time when the cursor leaves the center target radius
    
    Args:
        traj (ntrials list of (nt,2)): x,y trajectory data
        target_radius (float): the radius of the center target
        
    Returns:
        cursor_leave_center_time (ntrials list): the time when the cursor leaves the center target radius. Nan if cursor doesn't leave center target.
    '''
    ntr = len(data)
    cursor_leave_center_time = []
    leave_idx = []
    for itr in range(ntr):
        dist = np.sqrt(data[itr][:,0]**2 + data[itr][:,1]**2)
        
        try:
            temp_leave_idx = np.where(dist>target_radius)[0][0]
        except:
            temp_leave_idx = np.nan
        leave_idx.append(temp_leave_idx)
    
    return leave_idx

def smooth_timeseries_gaus(timeseries_data, samplerate, width, nstd=3, conv_mode='same'):
    '''
    Smooths across 2 
    
    Args:
        timeseries_data (ntime, ...)
        samplerate (int): Sample rate of timeseries
        width (float): Width of the gaussian in time [ms] from -nstd to +nstd
        nstd (float/int): Number of standard deviations to be used in the filter calculation.
        conv_mode (str): Sets the size of the output. Takes eithe 'full', 'valid', or 'same'. See scipy.signal.convolve for full documentationat
        
    Returns: 
        smoothed_timeseries
    '''
    sample_std = (width/nstd)*(samplerate/(1000)) # Convert from s to ms
    x = np.arange(-sample_std*nstd, nstd*sample_std+1)
    gaus_filter = (1/(sample_std*np.sqrt(2*np.pi)))*np.exp(-(x**2)/(2*sample_std**2))
    return np.apply_along_axis(scipy.signal.convolve, 0, timeseries_data, gaus_filter, mode=conv_mode, method='direct')

# Select relevant task entries

In [4]:
# Load neuropixel center-out task data
# Beignet
if subject == 'beignet':
    # bad_tes = [13152, 13153, 13154, 13155, 13156, 13102]
    bad_tes = [13152, 13153, 13154, 13155, 13156, 13272, 12290, 12291, 13102, 11971] # Also remove recording at site 48 (12290 & 12291)
    # bad_tes = [13102,13152, 13153, 13154, 13155, 13156, 13272, 9940, 9958, 10812, 10820, 12290, 12291]

# Affi
elif subject == 'affi':
    bad_tes = [11971, 11974, 11981, 11982, 11999, 12001, 12013, 12016, 12027, 12028, 12385, 12389, 12390, 12391, 12392, 12393, 12394, 12396, 12397,
              17294, 17296, 17297, 17299, 17301, 17302, 17303, 17304, 17305, 17316, 17318, 17319, 17547,17548, 17552, 17558, 12365, 12000, 
              18161, 18162, 18164, 18169, 18202, 18204,] 

mc_entries =  db.get_task_entries(subject__name=subject, task__name='manual control', date=(start_date, end_date))
mc_entries = [me for me in mc_entries if 'neuropixel_port1_drive_type' in me.task_params and me.task_params['neuropixel_port1_drive_type'] in implant_name
             and me.task_params['rotation']==task_coords and me.entry_name != 'flash']

# Remove bad TE IDs
mc_entries = [me for me in mc_entries if me.id not in bad_tes]
dates = np.unique([me.date.date() for me in mc_entries])

print(mc_entries, '\n','\n', dates)

[2024-05-30 10:16:10.257885: affi on manual control task, id=17536, 2024-05-31 09:46:01.684863: affi on manual control task, id=17542, 2024-05-31 10:19:02.740122: affi on manual control task, id=17543, 2024-06-02 09:37:43.847715: affi on manual control task, id=17553, 2024-06-03 08:49:00.861833: affi on manual control task, id=17556, 2024-06-04 09:12:52.159749: affi on manual control task, id=17560, 2024-06-05 09:28:39.226492: affi on manual control task, id=17568, 2024-06-06 09:40:49.027110: affi on manual control task, id=17571, 2024-06-07 09:06:58.232177: affi on manual control task, id=17574, 2024-08-28 09:36:13.293318: affi on manual control task, id=18110, 2024-08-29 09:59:03.643873: affi on manual control task, id=18128, 2024-08-30 09:13:04.119401: affi on manual control task, id=18136, 2024-09-02 08:45:27.849920: affi on manual control task, id=18166, 2024-09-03 09:44:33.503732: affi on manual control task, id=18170, 2024-09-04 10:05:24.581010: affi on manual control task, id=1

# Load behavioral data

In [5]:
start = time.time()
aopy.utils.release_memory_limit()
df, rasters, preproc_metadata = aopy.data.base.pkl_read(f"{subject}_np_preprocessed", os.path.join(base_save_dir, np_preproc_data_folder))
print(f"{np.round((time.time()-start)/60)} min to load preprocessed data")
nrecs = preproc_metadata['nrecs']
recording_site = preproc_metadata['recording_sites'] # will be the same for all align events
implants = ['NPinsert72' if preproc_metadata['implant'][irec] == 'NP_Insert72' else 'NPinsert137' for irec in range(len(preproc_metadata['implant']))] #Rename because name in bmi3d is slightly different (TODO)
dates = np.unique(df['date'])

7.0 min to load preprocessed data


In [6]:
# subjects = [subject for me in mc_entries]
# te_ids = [me.id for me in mc_entries]
# me_dates = [me.date.date() for me in mc_entries]
# df = aopy.data.bmi3d.tabulate_behavior_data_center_out(data_path_preproc, subjects, te_ids, me_dates, metadata=['target_radius'])
# df['reward'] = df['reward'].astype(bool)
# success_rate = aopy.analysis.calc_success_rate_trials(df['reward'], df['reach_completed'], window_size=success_rate_window)
# success_rate_date_labels = df['date']
# df = df[df['reward']].reset_index(drop=True)

# # Add cursor trajectories
# traj_times = np.array([(hst-tbefore_mc, e[-1]+tafter_mc) for hst, e in zip(df['hold_start_time'], df['event_times'])])
# df['cursor_traj'] = aopy.data.bmi3d.tabulate_kinematic_data(data_path_preproc, df['subject'], df['te_id'], df['date'], traj_times[:,0], traj_times[:,1], datatype='cursor')
# df['hand_traj'] = aopy.data.bmi3d.tabulate_kinematic_data(data_path_preproc, df['subject'], df['te_id'], df['date'], traj_times[:,0], traj_times[:,1], datatype='hand')
# df['start_time'] = traj_times[:,0]

# # Add behavior metrics
# df['duration'] = [(t[-1]-t[0])- (tbefore_mc+tafter_mc) for t in traj_times] 
# cursor_traj = [np.array(t) for t in df['cursor_traj']]
# hand_traj = [np.array(t) for t in df['hand_traj']]
# df['cursor_vel_traj'] = [np.array([aopy.utils.derivative(np.arange(len(t))/1000, t[:,0]), aopy.utils.derivative(np.arange(len(t))/1000, t[:,1])]).T for t in cursor_traj]
# df['cursor_vel'] =  [np.mean(aopy.utils.derivative(np.arange(len(t))/1000, t)) for t in cursor_traj]
# df['hand_vel_traj'] = [np.array([aopy.utils.derivative(np.arange(len(t))/1000, t[:,0]), aopy.utils.derivative(np.arange(len(t))/1000, t[:,1]), aopy.utils.derivative(np.arange(len(t))/1000, t[:,2])]).T for t in hand_traj]
# df['hand_vel'] =  [np.mean(aopy.utils.derivative(np.arange(len(t))/1000, t)) for t in hand_traj]

In [7]:
# # Get specs of loaded data
# reach_times = np.array(df['duration'][df['reward']])
# n_mctrials = [len(df[(df['date']==date)*df['reward']]) for date in dates]
# ntargets = len(np.unique(np.array(df['target_idx'][df['reward']])))
# unique_targets = aopy.data.bmi3d.get_target_locations(data_path_preproc, subject, df['te_id'][0], df['date'][0], np.unique(df['target_idx']))
# reach_time_thresh = np.median(np.hstack(reach_times)) + (np.median(np.hstack(reach_times))-np.min(np.hstack(reach_times)))
# good_trial_idx1 = df['duration'] <= reach_time_thresh # Labels for reach trials less than the max time (doesn't 

In [8]:
# # Define 'good_trial_idx' so that all targets from all penetrations have the same number of trials
# min_trials_to_target = np.min([np.min(np.unique(np.array(df['target_idx'])[df['date']==date][good_trial_idx1[df['date']==date]], return_counts=True)[1]) for dateidx, date in enumerate(dates)])
# ngood_trials = ntargets*min_trials_to_target
# df['good_trial'] = False
# good_trial_idx = []
# for idate, date in enumerate(dates):
#     good_trial_idx_temp = []    
#     [good_trial_idx_temp.extend(np.where(np.logical_and(df['target_idx'][df['date']==date]==itarget+1, good_trial_idx1[df['date']==date]))[0][:min_trials_to_target]) for itarget in range(ntargets)]
#     good_trial_idx_mask = np.zeros(n_mctrials[idate], dtype=bool)
#     good_trial_idx_mask[good_trial_idx_temp] = True
#     # df['good_trial'][df['date']==date] = good_trial_idx_mask
#     df.loc[df['date']==date, ['good_trial']] = good_trial_idx_mask
#     # df.loc[df['te_id']==me.id, ['recording_site']] = exp_metadata['neuropixel_port1_site']

In [9]:
# # All trials
# samplerate = 1000
# go_cue_idx = np.ceil((df['go_cue_time'] - df['start_time'])*samplerate).astype(int)
# trial_end_idx = np.ceil((df['reach_end_time'] - df['start_time'])*samplerate).astype(int)

In [10]:
# reach_time_list = [np.array(df['duration'][df['date']==date])[df['good_trial'][df['date']==date]] for date in dates]


# Load neuropixel spiking metadata

In [11]:
# ntime = np.round((tafter_mc + tbefore_mc)/spike_bin_width_mc).astype(int)
# trial_time_axis = np.arange(-tbefore_mc, tafter_mc, spike_bin_width_mc)
# ntargets = len(np.unique(df['target_idx']))

In [12]:
# # Concatenate trials across sessions within a day
# # TODO: concatenatmc_entries if recorded at the same stim site but during different days
# start = time.time()
# spike_times = []
# unit_labels = []
# trial_times = []
# spike_segs = []
# spike_align = []
# spike_align_raster = []
# spike_labels = []
# spike_pos = []
# ks_labels = []
# recording_site = []
# implant_name = []
# df['recording_site'] = 0
# df['implant_name'] = ''

# for ime, me in enumerate(tqdm(mc_entries)):

#     # Load data from sessions recorded on the same day and combine 
#     # Load data
#     exp_data, exp_metadata = aopy.data.load_preproc_exp_data(data_path_preproc, subject, me.id, me.date.date())
#     # filename_mc = aopy.data.get_preprocessed_filename(subject, me.id, me.date.date(), 'ap'
        
#     samplerate = exp_metadata['cursor_interp_samplerate']    
    
#     df.loc[df['te_id']==me.id, ['recording_site']] = exp_metadata['neuropixel_port1_site']
#     df.loc[df['te_id']==me.id, ['implant_name']] = exp_metadata['neuropixel_port1_drive_type']

In [13]:
# # Get spike_seg idx for relevant events (Also will need to get idx for kinematics)
# df['delay_start_kin_idx'] = np.ceil((df['delay_start_time'] - df['start_time'])*samplerate)
# df['delay_start_neural_idx'] = np.ceil((df['delay_start_time'] - df['start_time'])*(1/spike_bin_width_mc))
# df['go_cue_kin_idx'] = np.ceil((df['go_cue_time'] - df['start_time'])*samplerate)
# df['go_cue_neural_idx'] = np.ceil((df['go_cue_time'] - df['start_time'])*(1/spike_bin_width_mc))
# df['reach_end_kin_idx'] = np.ceil((df['reach_end_time'] - df['start_time'])*samplerate)
# df['reach_end_neural_idx'] = np.ceil((df['reach_end_time'] - df['start_time'])*(1/spike_bin_width_mc))
# df['mov_onset_kin_idx'] = 0
# df['mov_onset_neural_idx'] = 0

# # Calculate movement onset
# for ite in np.unique(df['te_id']): # Each TE has the same target radius
#     # Must start at delay_start_time 
#     traj = [temp_traj[np.array(go_cue_idx[df['te_id']==ite])[itraj]:,:] for itraj, temp_traj in enumerate(df['cursor_traj'][df['te_id']==ite])]
#     df.loc[df['te_id']==ite, ['mov_onset_kin_idx']] = np.array(get_cursor_leave_center_idx(traj, np.array(df['target_radius'][df['te_id']==ite])[0])) + np.array(go_cue_idx[df['te_id']==ite])
#     kin_neural_samplerate_ratio = samplerate/(1/spike_bin_width_mc)
#     df.loc[df['te_id']==ite, ['mov_onset_neural_idx']] = np.array(df['mov_onset_kin_idx'][df['te_id']==ite])//kin_neural_samplerate_ratio

# Load neuropixel LFP power

In [14]:
# df['recording_site'][(df['recording_site']==55)*(df['date']==dates[2])] = 56
# df['recording_site'][(df['recording_site']==55)*(df['date']==dates[4])] = 47
# # ks_combined_tes = {dates[0]: [9922, 9924, 9925],     # (2023, 7, 13)
# #                    dates[1]: [9928, 9929],           # (2023, 7, 14)
# #                    dates[2]: [9940],                 # (2023, 7, 18)
# #                    dates[3]: [9958],                 # (2023, 7, 19)
# #                    dates[4]: [10799, 10800, 10802],  # (2023, 8, 28)
# #                    dates[5]: [10810, 10812],         # (2023, 8, 29)
# #                    dates[6]: [10818, 10820],         # (2023, 8, 30)
# #                    dates[7]: [10824, 10827, 10828],  # (2023, 8, 31)
# #                    dates[8]: [10835],                # (2023, 9, 1)
# #                    dates[9]: [12269, 12270],         # (2023, 11, 16)
# #                    dates[10]: [13122],               # (2023, 12, 28)
# #                    dates[11]: [13239],               # (2023, 1, 3)
# #                    dates[12]: [13256],               # (2023, 1, 4)
# #                    dates[13]: [14116],               # (2023, 2, 1)
# #                    dates[14]: [14139, 14141]}        # (2023, 2, 2)

In [15]:
from datetime import date
# dtype = 'int16'
nch = 384
ks_folder_name_cutoff2 = date(2024, 8, 15)

aopy.utils.release_memory_limit()
lfp_spec_segs = []
for idate, date in enumerate(tqdm(dates[:])):
    # if date > ks_folder_name_cutoff:
    #     ks_preproc_path = os.path.join(data_path_preproc, f"kilosort/{date}_Neuropixel_ks_{subject}_site{list(df['recording_site'][df['date']==date])[0]}_bottom_port1")
    # else:
    #     ks_preproc_path = os.path.join(data_path_preproc, f"kilosort/{date}_Neuropixel_ks_{subject}_bottom_port1")
    
    # If all data is loaded from this day, go to next day
    nte_id = np.unique(df['te_id'][df['date']==date])
    try:
        for ite_id, te_id_temp in enumerate(nte_id):
            _ = aopy.data.base.pkl_read(f'lfp_power_te{te_id_temp}', lfp_power_save_dir)
            # apband_spec_segs.extend(apband_spec_segs_teid)
            print(f"LFP power data already exists from te{te_id_temp}")

        print(f"LFP power already exists for all TEs on {date}")
    except:
        # # Get whitening matrix
        # rez_path = os.path.join(ks_preproc_path, 'kilosort_output/rez.mat')
        # with h5py.File(rez_path, 'r') as f:
        #     whitening_matrix = np.array(f['rez']['Wrot'])

        # inv_wht_matrix = np.linalg.pinv(whitening_matrix)

        # # Load drift corrected and whitened apdata
        # drift_corrected_apdata_path = os.path.join(ks_preproc_path, 'temp_wh.dat')
        # data = np.memmap(drift_corrected_apdata_path, mode='r', dtype=dtype).reshape(-1,nch)

        # # Correct by multiplying ap data by inverse whitening matrix
        # corrected_data = data @ inv_wht_matrix
        # # corrected_data = data

        
        # Load raw apdata and check that it is the same length. If not, subselect the relevant data. This happens because data was concatenated before kilosort    
        # temp_data = []
        # if len(ks_combined_tes[date]) > 1:
        #     relevant_tes = np.unique(df['te_id'][df['date']==date])
        #     start_sample = 0
        #     for te_id_temp in ks_combined_tes[date]:
        #         data_folder_mc = f"{date}_Neuropixel_{subject}_te{te_id_temp}"
        #         rawdata_mc, rawmetadata = aopy.data.neuropixel.load_neuropixel_data(data_path_raw, data_folder_mc, 'ap')
        #         end_sample = start_sample + rawdata_mc.samples.shape[0]
        #         if te_id_temp in relevant_tes:
        #             temp_data.append(corrected_data[start_sample:end_sample,:])
        #         start_sample = end_sample

        # else:
        #     temp_data = [corrected_data]   
        # del corrected_data, data # Clear up memory
            # print(f"WARNING: Mismatch of samples between temp_wh.dat file and raw ap data -- please check")


    #     # Downsample AP band data --- this needs to handle multiple TEs
        for ite_id, te_id_temp in enumerate(nte_id):
            try:
                lfp_spec_segs_teid = aopy.data.base.pkl_read(f'lfp_power_te{te_id_temp}', lfp_power_save_dir)
                # apband_spec_segs.extend(apband_spec_segs_teid)
                print(f"LFP power data already exists from te{te_id_temp}")
            except:
                print(f"No LFP power data found for te{te_id_temp} - processing now....")
                preproc_filename_mc = aopy.data.get_preprocessed_filename(subject, te_id_temp, date, 'lfp')
                if date > ks_folder_name_cutoff2:
                    preproc_filename_mc = preproc_filename_mc[:-4] + '_port1.hdf'
                lfp_data_mc = aopy.data.load_hdf_group(os.path.join(data_path_preproc, subject), preproc_filename_mc, 'lfp')
                lfp_metadata_mc = aopy.data.load_hdf_group(os.path.join(data_path_preproc, subject), preproc_filename_mc, 'metadata')
                
                clean_lfp_data = (destripe_lfp((lfp_data_mc['lfp']*lfp_metadata_mc['bit_volts']).T/1e6, lfp_metadata_mc['sample_rate'], neuropixel_version=1, channel_labels=True)*1e6).T # convert back to uV

                # data_folder_mc = f"{date}_Neuropixel_{subject}_te{te_id_temp}"
                # rawdata_mc, metadata_mc = aopy.data.neuropixel.load_neuropixel_data(data_path_raw, data_folder_mc, 'ap')
                # corrected_apdata_dwns = aopy.precondition.base.downsample(temp_data[ite_id], metadata_mc['sample_rate'], samplerate_dwns)
                # ap_timestamps,_ = aopy.preproc.base.interp_timestamps2timeseries(lfp_data_mc['sync_timestamp'], lfp_data_mc['sync_timestamp'], samplerate=samplerate_dwns) # upsample lfp timestamps to AP

                # Trial align ap_timestamps for each TE and put into DF
                traj_times = np.array([(hst-tbefore_mc, e[-1]+tafter_mc) for hst, e in zip(np.array(df['hold_start_time'][df['te_id']==te_id_temp]), np.array(df['event_times'][df['te_id']==te_id_temp]))])
                ntrials_teid=traj_times.shape[0]
                talign_times_mc_segs = []
                talign_idx_mc_segs = []
                lfp_spec_segs_teid = []
                time_axes = []
                for itrial in tqdm(range(ntrials_teid)):
                    talign_times_mc, talign_idx_mc = aopy.preproc.base.trial_align_times(lfp_data_mc['sync_timestamp'], [traj_times[itrial,0]], 0, traj_times[itrial,1]-traj_times[itrial,0])
                    talign_times_mc_segs.append(talign_times_mc)
                    talign_idx_mc_segs.append(talign_idx_mc)
                
                    #filter
                    t, spec_temp = aopy.analysis.base.get_bandpower_feats(clean_lfp_data[talign_idx_mc[0],:], lfp_metadata_mc['sample_rate'], bands=bands, log=True, ref=False, 
                                                                n=n, p=p, k=k, fk=bands[-1][1], step = spike_bin_width_mc)
                    lfp_spec_segs_teid.append(spec_temp)
                    time_axes.append(t-tbefore_mc)
                
                lfp_power_metadata = {'samplerate': int(1/spike_bin_width_mc), 'frequency_band': bands, 'ch_ypos': lfp_metadata_mc['ypos'], 'ch_xpos': lfp_metadata_mc['xpos'], 'time_axis':time_axes}

                if save_lfp_power:
                    print(f"Saving LFP power data for te{te_id_temp}")
                    aopy.data.base.pkl_write(f'lfp_power_te{te_id_temp}', (lfp_spec_segs_teid, lfp_power_metadata), lfp_power_save_dir)
                    print("Saved.")

  0%|          | 0/26 [00:00<?, ?it/s]

LFP power data already exists from te11983
LFP power already exists for all TEs on 2023-10-31
LFP power data already exists from te12383
LFP power data already exists from te12386
LFP power already exists for all TEs on 2023-11-28
LFP power data already exists from te17536
LFP power already exists for all TEs on 2024-05-30
LFP power data already exists from te17542
LFP power data already exists from te17543
LFP power already exists for all TEs on 2024-05-31
LFP power data already exists from te17553
LFP power already exists for all TEs on 2024-06-02
LFP power data already exists from te17556
LFP power already exists for all TEs on 2024-06-03
LFP power data already exists from te17560
LFP power already exists for all TEs on 2024-06-04
LFP power data already exists from te17568
LFP power already exists for all TEs on 2024-06-05
LFP power data already exists from te17571
LFP power already exists for all TEs on 2024-06-06
LFP power data already exists from te17574
LFP power already exists 

  0%|          | 0/597 [00:00<?, ?it/s]

Saving LFP power data for te18291
Saved.
